In [5]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [30]:
from importlib import reload
import json
import utils

reload(utils)

chat = utils.chat

In [31]:
query_rewrite_prompt = """
    You are an expert in language and understanding semantic meaning.
    You read input from person and your job is to expand the input
    into stand alone atomic questions.

    You must return JSON using the template below.

    Example:
    Input: Who is Mick Jagger and how old is he?
    Output: 
    {
        "questions": [
            "Who is Mick Jagger?",
            "How old is Mick Jagger?"
        ]
    }
"""

def query_rewrite(input: str) -> list[str]:
        
        messages = [
            {"role": "system", "content": query_rewrite_prompt},
            {"role": "user", "content": f"The user question to rewrite: '{input}'"},
        ]
        config = {
            "response_format": {"type": "json_object"},
        }
        output = chat(messages, model = "gpt-3.5-turbo-0125", config=config, )
        try:
            return json.loads(output)["questions"]
        except json.JSONDecodeError:
            print("Error decoding JSON")
        return []


In [34]:
response = query_rewrite("Who directed the movie 'The Godfather', how long is it and what is it about?")
print(f"Query rewrite results: {response}")

Query rewrite results: ["Who directed the movie 'The Godfather'?", "How long is the movie 'The Godfather'?", "What is the movie 'The Godfather' about?"]
